## Introduction

*"The size of the credit-default-swap (CDS) market, which stood at a monumental \$57 trillion dollars in June 2008, has grabbed attention. That number has since shrunk dramatically. According to the Bank for International Settlements, the value of outstanding CDS contracts had fallen to \$42 trillion in December. The apparent collapse is largely down to something called trade compression."* (The Economist)

Trade compression is used to reduce the number of contracts that banks have on their books, while keeping the same economic exposure (present value, risks). 

- Unilateral basis: cancelling offsetting contracts in their own portfolio;
- Bilateral basis: firms cancel offsetting involving two parties;
- Multilateral basis: firms cancel offsetting contracts involving multiple parties.

![Multilateral](files/multilateral.png)


<font color='red'>**What is the economic motivation for performing compressions?**</font>
- Reducing the number of outstanding contracts (number of trades, total notional).
- Reduction of capital needed to cover trading book risk.
- Mitigating credit/counterparty risk.
- Easier to manage trading book and hedge with a smaller number of positions.
- The issue is even more acute with central clearing, as one position turns into two.

There are firms offering algorithms for compressing trades: TriOptima and CLS are the main market providers.

Compression effectiveness in greatly facilitated by the level of standardization of the underlying product. For instance, if maturities, coupons are standardized, as it is the case for CDS, then finding offsetting positions is more straightforward.

### Unilateral case

Compression is an optimization problem, where the task is to find a portfolio that is equivalent to the original one, while minimalizing a measure (e.g. number of trades). Equivalence could be defined with a set of boundary conditions:
- Present Value
- Credit Risk
Can be prescribed to remain constant.

In our case, the goal is to reduce the number of securities in a portfolio to minimize the computational cost for our simulations, while keeping these additive risk profiles constant:
- Current Market Price (MTM) of the securities;
- First and higher order derivatives of the market price.
The new portfolio will be a linear combination of the original.

Unfortunately, solving this is a NP-complete problem. Instead, we will investigate a related question that approximates this: **minimalizing the total absolute notional**.

In [12]:
import pandas as pd

trades = pd.DataFrame.from_csv('trades.csv')
trades

,Notional,MtM,1m,3m,6m,9m,1y,18m,2y,3y,4y,5y,6y,7y,8y,9y,10y,15y,20y,30y
ID,,,,,,,,,,,,,,,,,,,,
1,-1.862869,0.000278,-7.233350e-09,-2.771920e-08,4.484040e-04,4.730820e-04,-1.635010e-07,-1.564281e-03,-3.132020e-07,-4.059890e-07,1.110891e-03,1.139987e-03,-7.136930e-07,-0.001563,0.000321,-0.000010,0.054337,0.279816,0.000000,0.000000
2,3.264606,-0.000278,7.230340e-09,2.773060e-08,-4.484040e-04,-4.730820e-04,1.635240e-07,2.388860e-07,3.131870e-07,4.060410e-07,-3.660396e-02,-3.756553e-02,0.000000e+00,0.000000,-0.075368,0.000000,0.000000,0.000000,0.000000,0.000000
3,-32.784476,0.000275,-7.156300e-09,-2.747500e-08,-6.585520e-08,-1.110250e-07,1.113311e-03,-1.378500e-03,-3.104980e-07,-4.024900e-07,-5.053530e-07,-6.110680e-07,2.166197e-03,-0.001898,0.000276,-0.000023,0.152803,0.152181,0.000000,0.000000
4,23.924894,-0.000281,-1.889490e-04,-1.848960e-04,6.511500e-08,1.097760e-07,1.654210e-07,1.564410e-03,-1.183081e-03,-6.412440e-04,4.995850e-07,6.041160e-07,7.213590e-07,0.000605,-0.000308,-0.000026,-0.000028,0.000142,-0.184648,-0.319935
5,9.563576,-0.000278,7.263140e-09,2.779380e-08,-7.651640e-04,-7.349620e-05,1.638350e-07,4.197820e-04,3.137930e-07,4.068220e-07,-1.875583e-03,-3.294070e-04,7.147770e-07,-0.065458,-0.068358,-0.049801,0.000000,0.000000,0.000000,0.000000
6,2.551827,-0.000277,7.217410e-09,2.764820e-08,-1.785370e-03,-3.130087e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.033363,0.000000,0.000000,0.000000,0.000000,0.000000
7,-45.722538,0.000278,-7.234370e-09,-2.772450e-08,4.484040e-04,4.730820e-04,-1.634990e-07,-1.564281e-03,-3.132030e-07,-4.060110e-07,1.110891e-03,1.139987e-03,-7.136390e-07,-0.001563,0.000321,0.130621,0.080100,0.000000,0.000000,0.000000
8,-2.216915,0.000278,-7.233350e-09,-2.771920e-08,4.484040e-04,4.730820e-04,-1.635010e-07,-1.564281e-03,-3.132020e-07,-4.059890e-07,1.110891e-03,1.139987e-03,-7.136930e-07,-0.001563,0.000321,-0.000010,0.054337,0.279816,0.000000,0.000000
9,9.142990,-0.000280,-4.632440e-05,-4.534190e-04,6.492450e-08,1.095120e-07,1.649930e-07,1.564257e-03,-2.264070e-04,-1.742499e-03,4.981850e-07,6.023620e-07,7.192610e-07,0.000850,-0.000332,-0.156064,-0.043065,0.000000,0.000000,0.000000


**Definitions:**

- We have $𝑁$ trades in a portfolio.
- Let’s denote the vector of nationals corresponding to these by $ \bf{c} = \left [ c_1, ..., c_N \right ] $.
- We have $𝑀$ risk factors for each trade, e.g. market price (MtM) and it’s derivatives for different tenors.
- In matrix $\bf{A}$ for the $i$th trade risk factor $j$ is denoted by $ A_{i,j} $.
- The sum of the risk factors over the whole portfolio is: $ \bf{\Sigma} = \left [ \Sigma_1, ..., \Sigma_M \right ] $, where $ \Sigma_j = \sum_{i=1}^{N} {A_{i,j}}$.

We seek a vector of portfolio weights $ \bf{w} = \begin{bmatrix} w_1 \\ ... \\ w_N \end{bmatrix} $ with the boundary condition:
$$ \forall i: 0 \leq w_i \leq w_{max} $$
$$ \bf{\Sigma} + \bf{L} \leq \bf{A}^T \bf{w} \leq \bf{\Sigma} + \bf{U} , $$
where $\bf{L}$ and $\bf{U}$ are tolerance levels defined by the user.

In [ ]:
Example: w, L, U, new

## Problem as linear programming

Minimalizing the total notional under boundary conditions:

$$ \bf{w}: \mathop {\arg \min}\limits_{\bf{w}} {\bf{c}^T \bf{w}} $$
such that
$$ \forall i: 0 \leq w_i \leq w_{max} $$
$$ \bf{\Sigma} + \bf{L} \leq \bf{A}^T \bf{w} \leq \bf{\Sigma} + \bf{U} $$
or in a canonical form
$$ \begin{bmatrix} \bf{A}^T \\ - \bf{A}^T \end{bmatrix} \bf{w} \leq \begin{bmatrix} \bf{\Sigma} + \bf{U} \\ - \left(\bf{\Sigma} + \bf{L}\right) \end{bmatrix} $$

<font color='red'>Such task is called LP optimization problem.</font>

**Properties:**
- Initial solution: $\bf{𝐰}=\bf{𝟏}$ (not optimal, but feasible)
- Solution is not unique
- Solution tends to be sparse, i.e. the number of non-zero elements are small

### How to solve LP problems – Simplex algorithm

**Graphical method:**
1. Construct the smallest convex hull based on the constraints
2. Walk on the edges towards increasing/decreasing objective function (greedy algorithm, too many vertices for exhaustive search)

2-D | N-D
- | -
![2-D](files/simplex_2d.png) | ![N-D](files/simplex_nd.png)

**Properties:**
- Global optimum is guaranteed
- Usually efficient, worst-case exponential
- Listed as one of the top 10 algorithms in the 20th century (by IEEE)

#### Scipy implementation

```python
scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='simplex', callback=None, options={'disp': False, 'bland': False, 'tol': 1e-12, 'maxiter': 1000})
```

- c = $ \bf{c} $
- A_ub = $ \begin{bmatrix} \bf{A}^T \\ - \bf{A}^T \end{bmatrix} $
- b_ub = $ \begin{bmatrix} \bf{\Sigma} + \bf{U} \\ - \left(\bf{\Sigma} + \bf{L}\right) \end{bmatrix} $
- bounds = $ \forall i: \left ( 0, w_{max} \right ) $

Returns: a ```scipy.optimize.OptimizeResult```
- x = $ \bf{w} $
- fun = $ {\bf{c}^T \bf{w}} $
- success: True if the algorithm succeeded in finding an optimal solution

More details: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/optimize.linprog-simplex.html)

In [13]:
from scipy.optimize import linprog
import numpy as np

In [131]:
notional = [750, 1700, 7850, 590, 1340, 3240, 800, 750, 4820, 3385, 3385, 3385, 620, 1800, 4900, 755, 3400, 600, 1228, 2179]

w_max = 3
A = np.transpose(np.multiply(np.transpose(A), notional))
notional = [1]*20

In [100]:
N = len(A)
M = len(A[1])
buckets = np.sum(A, axis=0) # np.dot(np.transpose(A), notional)
tolerance_lower = np.append([-100000], [-200] * (M-1))
tolerance_upper = np.append([100000], [200] * (M-1))

In [134]:
options = {'disp': True, 'bland': False, 'tol': 1e-10, 'maxiter': 10000}
A_ub = np.append(np.transpose(A), -np.transpose(A), axis=0)
b_ub = np.append(buckets + tolerance_upper, -(buckets + tolerance_lower), axis=0)
bounds=[(0, w_max)] * N
solution = linprog(notional, A_ub=A_ub, b_ub=b_ub, bounds=bounds, options=options, method='simplex')
w = solution.x

Optimization terminated successfully.
         Current function value: 3.067909    
         Iterations: 14


In [115]:
np.dot(A_ub, w) <= b_ub

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [132]:
print "Compression (#trades): %1.2f%%" %(100.0*np.count_nonzero(w)/N)
print "Compression (notional): %1.2f%%" %(100.0*np.dot(notional, w)/np.dot(notional, notional))

Compression (#trades): 35.00%
Compression (notional): 25.61%


### Other LP algorithms

**Interior point methods:**
- Can visit any point in the feasible region, not only vertices.
- Similar efficiency as the simplex algorithm, depends on the actual problem.

**Approximate algorithms:**  solution is 𝑂(1+𝜀) optimal [1]
Matrix A is 𝑛 x 𝑚, and has N non-zero entries
Best sequential takes 𝑂(𝑁+(𝑙𝑜𝑔𝑁)(𝑛+𝑚) 𝜀^(−2) ) time
Best parallel takes 𝑂((𝑙𝑜𝑔𝑁)^2 𝜀^(−3) ) time

## Summary

- Trade compression is therefore an important means of reducing gross notional amounts
- For achieving regulatory capital savings; and reducing operational and counterparty risk exposures. 
- When used on a multilateral basis, for example, with cleared OTC derivatives trades, it also cuts back on double counting of risk. 
- The unilateral trade compression saved a lot of computational resources when calculating CVA.
- Therefore the importance of trade compression should not be underestimated.

In [ ]:
# http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html